# Exercise 7
## Justin Caniglia

In [10]:
import numpy as np
import pandas as pd
import os

In [13]:

#1. Use Pandas to repeat the Exercise 5.7.¶
#Hint 1: Use pd.read_csv(url, ...) to read the USGS online gauge data.

#Hint 2: Use df.set_index() to set the datetime column and df.resample() to calculate the monthly and annual statistics

myGageNums = ['06803495', '06803486']
myDates = ['2000-10-10', '2019-09-30']


In [14]:
def myURL(gageList, Dates):
    gages = ('&site_no={}' * len(gageList)).format(*gageList)
    period = f'&period=&begin_date={Dates[0]}&end_date={Dates[1]}'
    return f'https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb{gages}&referred_module=sw{period}'

In [15]:
raw_df = pd.read_csv(myURL(myGageNums, myDates), skiprows=29, delimiter='\t', header=0, names=['Agency','Site','Date','Discharge_cfs','QualCode'])

raw_df.head(20)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12315 entries, 0 to 12314
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Agency         12315 non-null  object
 1   Site           12306 non-null  object
 2   Date           12306 non-null  object
 3   Discharge_cfs  12306 non-null  object
 4   QualCode       12306 non-null  object
dtypes: object(5)
memory usage: 481.2+ KB


In [16]:
df = raw_df[(raw_df['Agency']=='USGS')].copy()
df.head(20)

Agency      Site        Date Discharge_cfs QualCode
0    USGS  06803486  2000-10-10          10.0        A
1    USGS  06803486  2000-10-11          10.0        A
2    USGS  06803486  2000-10-12          11.0        A
3    USGS  06803486  2000-10-13          12.0        A
4    USGS  06803486  2000-10-14          13.0        A
5    USGS  06803486  2000-10-15          11.0        A
6    USGS  06803486  2000-10-16          11.0        A
7    USGS  06803486  2000-10-17          11.0        A
8    USGS  06803486  2000-10-18          11.0        A
9    USGS  06803486  2000-10-19          11.0        A
10   USGS  06803486  2000-10-20          10.0        A
11   USGS  06803486  2000-10-21          9.60        A
12   USGS  06803486  2000-10-22          12.0        A
13   USGS  06803486  2000-10-23          13.0        A
14   USGS  06803486  2000-10-24          14.0        A
15   USGS  06803486  2000-10-25          16.0        A
16   USGS  06803486  2000-10-26          15.0        A
17   USGS  06803486  2000-10-27          14.0        A
18   USGS  06803486  2000-10-28          16.0        A
19   USGS  06803486  2000-10-29          17.0        A

In [18]:
dirPath = r'C:\Users\canig\Desktop\NRES 898\HW\Exercises\Exercise7'
fileName = 'USGS_Streamflow_Data.csv'
fullPath = os.path.join(dirPath, fileName)
print(fullPath)

C:\Users\canig\Desktop\NRES 898\HW\Exercises\Exercise7\USGS_Streamflow_Data.csv


In [ ]:
df.to_csv(fullPath, index=False)

In [ ]:
df['Date']= pd.to_datetime(df['Date'])

df.info()

In [ ]:
df = df.set_index(['Date'])

In [ ]:
df['Discharge_cfs'] = pd.to_numeric(df['Discharge_cfs'], errors='coerce')

In [ ]:
monthlyStats = pd.DataFrame(columns=['Min','Max','Mean'])

monthlyStats.Min = df.Discharge_cfs.groupby(df['Site']).resample('M').min()
monthlyStats.Max = df.Discharge_cfs.groupby(df['Site']).resample('M').max()
monthlyStats.Mean = df.Discharge_cfs.groupby(df['Site']).resample('M').mean()

In [ ]:
monthlyStats.head(20)

In [ ]:
fileName = 'monthlyStreamflowStats.csv'
fullPath = os.path.join(dirPath, fileName)

print(fullPath)

In [ ]:
df['acreFeetPerDay'] = df['Discharge_cfs']*24*60*60/43560

yearlyTotal = pd.DataFrame(columns=['acreFeet'])
yearlyTotal.acreFeet = df.acreFeetPerDay.groupby(df['Site']).resample('A').sum()
yearlyTotal

In [ ]:
fileName = 'annualAcreFeet.csv'
fullPath = os.path.join(dirPath, fileName)

print(fullPath)

# save fitered dataframe to .csv file
yearlyTotal.to_csv(fullPath, float_format='%.2f')

In [21]:
#2. Use Pandas to¶
#Read data in users.zip (using Pandas' the on-the-fly decompression cabability)

#Identify all the occupations and compare the user numbers between STEM-related occupations and non-STEM occupations.

#Identify the locations of the users that are programmers and above 35. How many male and female programmers, respectively? Is this ratio the same for the age under 35?

#Compare the numbers of male and female for each occupations

#Find the occupations with the youngest and oldest mean ages, respectively

#Based on the first two digits of the zip codes, find the area with the largest number of users

import pandas as pd
import os

dirPath = r'C:\Users\canig\Desktop\NRES 898\HW\Exercises\Exercise7'     #dirpath = r'C:\Users\canig\Desktop\NRES 898\HW\Exercises\Exercise7'
fileName = 'users.zip'
fullPath = os.path.join(dirPath, fileName)

In [22]:
userData = pd.read_csv(fullPath, compression= 'zip', sep='|')
userData = userData.set_index('user_id')

userData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 1 to 943
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   age         943 non-null    int64 
 1   gender      943 non-null    object
 2   occupation  943 non-null    object
 3   zip_code    943 non-null    object
dtypes: int64(1), object(3)
memory usage: 36.8+ KB


In [25]:
stem_List = ('Engineer','Programmer','Scientist','Technician')

stemTotal = 0
for item in stem_List:    
    num = len(userData[userData["occupation"] == item])
    print(f'{item} = {num}')
    stemTotal += num

totalAll = len(userData.index)
nonStemTotal = totalAll - stemTotal

print('Total amount is = ', totalAll)  
print('Total STEM is = ', stemTotal)
print('Total Non-STEM is = ', nonStemTotal)

Engineer = 0
Programmer = 0
Scientist = 0
Technician = 0
Total amount is =  943
Total STEM is =  0
Total Non-STEM is =  943


In [26]:
programmerAgeHigh = userData.query('occupation == "programmer" & age > 35').groupby(userData['gender']).count()
maleProgrammersAgeHigh = programmerAgeHigh['gender']['M']
femaleProgrammersAgeHigh = programmerAgeHigh['gender']['F']
programmersAgeHighRatio = f'{maleProgrammersAgeHigh/femaleProgrammersAgeHigh:.2f}'

print(f"number of male programmers over 35 = {maleProgrammersAgeHigh}")
print(f"number of male programmers over 35 = {femaleProgrammersAgeHigh}")
print(f"ratio of male to femail programmers over 35 = {programmersAgeHighRatio}\n")


programmerAgeLow = userData.query('occupation == "programmer" & age <= 35').groupby(userData['gender']).count()
maleProgrammersAgeLow = programmerAgeLow['gender']['M']
femaleProgrammersAgeLow = programmerAgeLow['gender']['F']
programmersAgeLowRatio = f'{maleProgrammersAgeLow/femaleProgrammersAgeLow:.2f}'

print(f"number of male programmers 35 & under= {maleProgrammersAgeLow}")
print(f"number of male programmers 35 & under = {femaleProgrammersAgeLow}")
print(f"ratio of male to femail programmers 35 & under = {programmersAgeLowRatio}\n")

number of male programmers over 35 = 19
number of male programmers over 35 = 2
ratio of male to femail programmers over 35 = 9.50

number of male programmers 35 & under= 41
number of male programmers 35 & under = 4
ratio of male to femail programmers 35 & under = 10.25



In [36]:
programmerOccupationStatistics = userData.groupby(['occupation','gender'])['gender'].count()
print('programmerOccupationStatistics')

programmerOccupationStatistics


In [34]:
meanAges = userData.groupby(['occupation'])['age'].mean()
print(meanAges.sort_values().round(decimals=2),'\n')

print(f'{meanAges.idxmin()} is the occupation with the youngest mean age of {meanAges.min():.1f} years')
print(f'{meanAges.idxmax()} is the occupation with the oldest mean age of {meanAges.max():.1f} years\n')

occupation
student          22.08
none             26.56
entertainment    29.22
artist           31.39
homemaker        32.57
programmer       33.12
technician       33.15
other            34.52
scientist        35.55
salesman         35.67
writer           36.31
engineer         36.39
lawyer           36.75
marketing        37.62
executive        38.72
administrator    38.75
librarian        40.00
healthcare       41.56
educator         42.01
doctor           43.57
retired          63.07
Name: age, dtype: float64 

student is the occupation with the youngest mean age of 22.1 years
retired is the occupation with the oldest mean age of 63.1 years



In [35]:
userData['shortZip'] = userData['zip_code'].str[:2]
numUsers = userData.groupby(['shortZip'])['occupation'].count()
print(f"the largest number of users is in zip code prefix '{numUsers.idxmax()}'")

the largest number of users is in zip code prefix '55'
